In [2]:
import sys
sys.path.insert(0, r'D:\Projects\aaa')

import pandas as pd
import matplotlib.pyplot as plt
import random
random.seed(42)

from src.utils.utils import get_files_in_from_directory

In [3]:
all = r'D:\Projects\aaa\results\rq4_results\features.csv'
npm = r'D:\Projects\aaa\results\rq4_results\features_npm.csv'
pypi = r'D:\Projects\aaa\results\rq4_results\features_pypi.csv'
mvn = r'D:\Projects\aaa\results\rq4_results\features_mvn.csv'

df_all = pd.read_csv(npm)

In [4]:
# Sanitize time_to_next_merge where nan is a valid value
max_merge = df_all['time_to_next_merge'].max() * 10
df_all.loc[df_all['time_to_next_merge'].isna(),'time_to_next_merge'] = max_merge

df_all.fillna(0, inplace=True)
df_all.fillna(0, axis=1, inplace=True)

In [5]:
print(df_all.shape[0])
print((df_all['label_security_related']==True).sum())

33336
561


In [6]:
_byrepo = df_all.groupby('label_repo_full_name')
repos = df_all['label_repo_full_name'].unique()
train_repos = random.sample(list(repos), int(0.9*len(repos)))
df = df_all[df_all.apply(lambda x: x['label_repo_full_name'] in train_repos, axis=1)]
eval_df = df_all[df_all.apply(lambda x: x['label_repo_full_name'] not in train_repos, axis=1)]

print(df.shape, (df['label_security_related']==True).sum())
print(eval_df.shape, (eval_df['label_security_related']==True).sum())


(30028, 161) 516
(3308, 161) 45


In [7]:
# nan_ratio = {col:0 for col in df}

# for col in df:
#     nan_counts = df[col].isna().sum()
#     nan_ratio[col] = nan_counts/df.shape[0]

# nan_ratio = {k: v for k,v in sorted(nan_ratio.items(), key=lambda kv: -kv[1])}
# for k,v in nan_ratio.item():
#     if v > 0.1:
#         print(nan_ratio)


In [8]:
broken_features = [
    'commits_to_next_merge',
    'commits_since_last_merge',
    'commits_to_next_merge'   
    'commits_since_last_merge'
]

In [9]:
highly_correlated_features = [
    'dmm_unit_complexity',
    'secur_in_title',
    'vulnerab_in_title',
    'exploit_in_title',
    'certificat_in_title',
    'authent_in_title',
    'leak_in_title',
    'sensit_in_title',
    'crash_in_title',
    'attack_in_title',
    'deadlock_in_title',
    'segfault_in_title',
    'malicious_in_title',
    'corrupt_in_title',
    'test_in_filename',
    'removed_lines_count_avg',
    'added_lines_count_avg',
    'changed_lines_count_avg',
    'modified_lines_count_avg',
    'file_size_avg',
    'changed_methods_count_max',
    'total_methods_count_avg',
    'max_method_token_count_avg'
    'avg_method_complexity_avg',
    'avg_method_complexity_max',
    'avg_method_parameter_count_max'

    'changes_to_file_in_prev_50_commits_avg',
    'changes_to_file_in_next_50_commits_avg'
]

In [10]:

df_selected = df
df_selected = df_selected[df_selected.columns.difference(broken_features)]
df_selected = df_selected[df_selected.columns.difference(highly_correlated_features)]

X = df_selected[df_selected.columns.difference(['label_repo_full_name', 'label_sha', 'label_commit_date', 'label_security_related'])]
y = df_selected['label_security_related']


In [11]:
from sklearn.feature_selection import VarianceThreshold
_Variance_Threshold = 0.95
sel = VarianceThreshold(threshold=(_Variance_Threshold * (1 - _Variance_Threshold)))
sel.fit(X)
sup = sel.get_support()
i = -1
for x in X:
       i+=1
       if not sup[i]:
              print('reject', x)


reject attack_in_file_content
reject attack_in_message
reject attack_in_patch
reject authent_in_message
reject authent_in_patch
reject authored_by_bot
reject certificat_in_file_content
reject certificat_in_message
reject certificat_in_patch
reject committed_by_bot
reject corrupt_in_file_content
reject corrupt_in_message
reject corrupt_in_patch
reject crash_in_file_content
reject crash_in_message
reject crash_in_patch
reject deadlock_in_file_content
reject deadlock_in_message
reject deadlock_in_patch
reject exploit_in_file_content
reject exploit_in_message
reject exploit_in_patch
reject has_mvn_code
reject has_mvn_like_code
reject has_npm_like_code
reject has_pypi_code
reject has_pypi_like_code
reject is_add
reject is_delete
reject is_file_recently_added
reject is_file_recently_removed
reject is_rename
reject leak_in_file_content
reject leak_in_message
reject leak_in_patch
reject malicious_in_file_content
reject malicious_in_message
reject malicious_in_patch
reject methods_with_attack_c

In [12]:
correlation_matrix = X.corr()
corr = correlation_matrix.values
column_names = correlation_matrix.columns

for i in range(len(column_names)):
    for j in range(i+1, len(column_names)):
        if abs(corr[i,j])> 0.75:
            print('reject', column_names[i], ' ', column_names[j], ' ', corr[i,j])

reject added_lines_ratio_avg   added_lines_ratio_max   0.7669421708804874
reject added_lines_ratio_avg   removed_lines_ratio_avg   -0.9791665820539546
reject authored_by_bot   committed_by_bot   0.801844239979437
reject avg_method_complexity_avg   max_method_complexity_avg   0.8163180592178367
reject avg_method_parameter_count_avg   avg_method_parameter_count_max   0.7661911738784531
reject avg_method_parameter_count_avg   max_method_parameter_count_avg   0.7730738670483328
reject changed_lines_count_max   removed_lines_count_max   0.8402062364572621
reject changed_methods_count_avg   file_changed_method_count_avg   0.9999999999996892
reject changes_to_file_in_next_50_commits_max   changes_to_file_in_prev_50_commits_max   0.8823266377533843
reject changes_to_file_in_prev_50_commits_avg   changes_to_file_in_prev_50_commits_max   0.834196558661542
reject file_complexity_avg   file_complexity_max   0.8148115448159027
reject file_complexity_avg   file_token_count_avg   0.9739823682243952
r

In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFECV
from sklearn.model_selection import KFold
from sklearn.model_selection import RandomizedSearchCV
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

VERBOSE_LVL = 10
n_iter = 15

sampler = RandomUnderSampler()
X_resampled, y_resampled = sampler.fit_resample(X, y)


model = RandomForestClassifier(
    n_estimators =100,
    random_state=42)
    

selector = RFECV(model, 
    step=1, 
    cv=5,
    min_features_to_select=40,
    scoring = 'f1',

    verbose=VERBOSE_LVL)
selector = selector.fit(X_resampled, y_resampled)
model = selector.estimator_
print(selector.support_)


Fitting estimator with 131 features.
Fitting estimator with 130 features.
Fitting estimator with 129 features.
Fitting estimator with 128 features.
Fitting estimator with 127 features.
Fitting estimator with 126 features.
Fitting estimator with 125 features.
Fitting estimator with 124 features.
Fitting estimator with 123 features.
Fitting estimator with 122 features.
Fitting estimator with 121 features.
Fitting estimator with 120 features.
Fitting estimator with 119 features.
Fitting estimator with 118 features.
Fitting estimator with 117 features.
Fitting estimator with 116 features.
Fitting estimator with 115 features.
Fitting estimator with 114 features.
Fitting estimator with 113 features.
Fitting estimator with 112 features.
Fitting estimator with 111 features.
Fitting estimator with 110 features.
Fitting estimator with 109 features.
Fitting estimator with 108 features.
Fitting estimator with 107 features.
Fitting estimator with 106 features.
Fitting estimator with 105 features.
F

In [16]:
from sklearn.metrics import recall_score, precision_score

eval_df_selected = eval_df
eval_df_selected = eval_df_selected[eval_df_selected.columns.difference(broken_features)]
eval_df_selected = eval_df_selected[eval_df_selected.columns.difference(highly_correlated_features)]

eval_X = eval_df_selected[eval_df_selected.columns.difference(['label_repo_full_name', 'label_sha', 'label_commit_date', 'label_security_related'])]
eval_y = eval_df_selected['label_security_related']


eval_y_pred = model.predict(selector.transform(eval_X))

recall = recall_score(eval_y, eval_y_pred)
precision = precision_score(eval_y, eval_y_pred)
print(recall, precision)

0.6 0.025738798856053385
